# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [160]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start} \n')
        return result

    return wrapper

## Задание 2

In [161]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        res_args = tuple(args)
        res_kwargs = {k:v for k,v in kwargs}
        print(f'Функция вызвана с параметрами: \n {res_args}, {res_kwargs} \n')
        return result

    return wrapper

## Задание 3

In [162]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    cnt = 0
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        nonlocal cnt
        cnt += 1
        if cnt%10 in [2,3,4] and cnt not in [12,13,14]:
            print(f'Функция была вызвана: {cnt} раза \n')
        else:
            print(f'Функция была вызвана: {cnt} раз \n')
        return result

    return wrapper

## Задание 4

In [134]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  def fmemo(*args):
        nonlocal cache
        for arg in args:
            for i in range(1, arg + 1):
                if i not in cache:
                    cache[i] = func(i)

  fmemo.cache = cache
  return fmemo

## Тестирование

In [170]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз "

print(word_count('whole'))

Время выполнения функции word_count: 1.1278350350003166 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 1 раз 

Cлово whole встречается 176 раз 


In [171]:
# проверяем счетчик запусков функции
for _ in range(4):
    print(word_count('whole'))
    print('----------------------------------')

Время выполнения функции word_count: 1.1505572509995545 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 2 раза 

Cлово whole встречается 176 раз 
----------------------------------
Время выполнения функции word_count: 1.337696168999173 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 3 раза 

Cлово whole встречается 176 раз 
----------------------------------
Время выполнения функции word_count: 1.1180554090005899 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 4 раза 

Cлово whole встречается 176 раз 
----------------------------------
Время выполнения функции word_count: 1.1089118259988027 

Функция вызвана с параметрами: 
 ('whole',), {} 

Функция была вызвана: 5 раз 

Cлово whole встречается 176 раз 
----------------------------------


In [76]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [146]:
# измеряем время выполнения
res = benchmark(fib)
res(35)
res(35)

Время выполнения функции fib: 3.786984192998716
Время выполнения функции fib: 4.7578634720011905


9227465

In [142]:
@memo
def fib2(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [143]:
# измеряем время выполнения
res2 = benchmark(fib2)
res2(35)
res2(35)
# при первом вызове функции составляется словарь значений функции с ключами от 1 до переданного значения
# при последующих запусках результат берется из полученного словаря и время на вычисление не требуется

Время выполнения функции fmemo: 10.611988593000206
Время выполнения функции fmemo: 5.983998562442139e-06
